### Import Packages

In [1]:
# geospatial analysis
import geopandas as gpd
import restapi # downloading data
from scipy.spatial import cKDTree
from pyproj import Proj

# working with data
import numpy as np
import h5pyd
import pandas as pd
import xarray as xr

# web scraping
import requests
from bs4 import BeautifulSoup
import time
import re

# API
from gridstatusio import GridStatusClient

# working with files
import os

# work with time
import pytz
import dateutil
from datetime import timezone

# helpful functions
from download_utils import *

# API key
GRIDSTATUS_API_KEY = os.environ["GRIDSTATUS_API_KEY"]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/restapi/common_types.py:42: UserWarning: No Arcpy found, some limitations in functionality may apply.
  warnings.warn('No Arcpy found, some limitations in functionality may apply.')


## Import Common Geometries

In [2]:
# California counties
us_counties = gpd.read_file("Data/Boundaries/cb_2018_us_county_500k/cb_2018_us_county_500k.shp")
ca_counties = us_counties[us_counties["STATEFP"] == "06"]
ca_counties.to_crs("EPSG:5070", inplace=True)

# California HUC8 subbasins
huc8_ca = gpd.read_file("Data/Boundaries/ca_HUC8.geojson")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## Electric Grid Headroom

### Downloading SCE

Other relevant links for Southern California Edison:

https://drpep-sce2.opendata.arcgis.com/maps/f76174d459a14545b2b13110d1cc6fba/about

Tutorial: https://drpep.sce.com/drpep/cdn/19/drpep-interactive-user-guide/index.html#/

In [2]:
# download Southern California Edison data
# sce_url = "https://drpep.sce.com/arcgis_server/rest/services/Hosted/AVL_LOAD_CAP_TOGGLE/FeatureServer/1"
sce_url = "https://drpep.sce.com/arcgis_server/rest/services/Hosted/AVL_LOAD_CAP_TOGGLE/FeatureServer/0"
sce_layer = restapi.FeatureLayer(sce_url)
# sce_layer.export_layer("Data/Hosting_Capacity/sce.geojson")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/restapi/rest_utils.py:42: UserWarning: no request client has been set, using default client
  warnings.warn('no request client has been set, using default client')


In [34]:
sce_layer.list_fields()

['circuit_name',
 'substation_name',
 'date_last_update',
 'plan_year',
 'system_name',
 'a_bank',
 'alc_year1',
 'alc_year2',
 'alc_year3',
 'alc_year4',
 'alc_year5',
 'subst_alc_year1',
 'subst_alc_year2',
 'subst_alc_year3',
 'subst_alc_year4',
 'subst_alc_year5',
 'a_bank_alc_year1',
 'a_bank_alc_year2',
 'a_bank_alc_year3',
 'a_bank_alc_year4',
 'a_bank_alc_year5',
 'circuit_voltage',
 'org_alc_year1',
 'org_alc_year2',
 'org_alc_year3',
 'org_alc_year4',
 'org_alc_year5',
 'ckt_capacity_y1',
 'ckt_capacity_y2',
 'ckt_capacity_y3',
 'ckt_capacity_y4',
 'ckt_capacity_y5',
 'subst_capacity_y1',
 'subst_capacity_y2',
 'subst_capacity_y3',
 'subst_capacity_y4',
 'subst_capacity_y5',
 'subst_org_alc_year1',
 'subst_org_alc_year2',
 'subst_org_alc_year3',
 'subst_org_alc_year4',
 'subst_org_alc_year5',
 'subtrans',
 'subtrans_alc_year1',
 'subtrans_alc_year2',
 'subtrans_alc_year3',
 'subtrans_alc_year4',
 'subtrans_alc_year5',
 'y1ckt_f',
 'y2ckt_f',
 'y3ckt_f',
 'y4ckt_f',
 'y5ckt_f'

In [28]:
cursor = restapi.SearchCursor(sce_layer, ["objectid"])

In [ ]:
# verify SCE data downloaded properly
sce_alc = gpd.read_file("Data/Hosting_Capacity/SCE/Available Load Capacity.geojson")

# for simplicity, let's say year 1 (2024)
sce_alc.drop(columns=sum([[f"alc_year{i}", f"a_bank_alc_year{i}", f"org_alc_year{i}", f"ckt_capacity_y{i}", f"subst_capacity_y{i}", f"subst_org_alc_year{i}", f"subtrans_alc_year{i}", f"y{i}ckt_f", f"y{i}sub_f", f"y{i}subt_f", f"y{i}abank_f", f"y{i}ckt_sub_f", f"y{i}ckt_subt_f", f"y{i}ckt_abank_f", f"y{i}sub_subt_f", f"y{i}sub_abank_f", f"y{i}subt_abank_f", f"y{i}ckt_sub_subt_f", f"y{i}ckt_sub_abank_f", f"y{i}ckt_subt_abank_f", f"y{i}sub_subt_abank_f", f"y{i}ckt_sub_subt_abank_f"] for i in range(2,6)], []), inplace=True)

# sce_alc.to_file("Data/SCE_available_load_capacity.geojson", driver="GeoJSON") # somehow the file size becomes bigger after this

### Downloading PGE

In [20]:
# verify PG&E data download properly
pge_ica = gpd.read_file("Data/Hosting_Capacity/PG&E/ICA_Results/LineDetail.shp")

### Downloading SDG&E

In [2]:
# verify SDG&E data download properly
sdge_ica = gpd.read_file("Data/Hosting_Capacity/SDG&E/ICA_MAP_PROD_LoadCapacityGrids_VW_-864651790911740139.geojson")

In [3]:
sdge_ica

,OBJECTID,JOIN_COUNT,TARGET_FID,JOIN_FID,SHAPE_LENG,VOLTAGE,LINE_SEGMENT_NUMBER,ICAWOF_UNIGENERATION,ICAWOF_UNILOAD,ICAWOF_PVGENERATION,OHUG,CIRCUIT_NAME,LABELTEXT,ICAWNOF_UNIGENERATION,ICAWNOF_UNILOAD,ICAWNOF_PVGENERATION,LABELTEXT_LCA,RESTRICTED,SUBID,geometry
0,28922611,None,None,None,None,12,141921,0.0,0.8,0.2,OH,456,None,0.0,0.8,0.5,Up To 1.00,N,ASH,"POLYGON ((-117.06446 33.13968, -117.06446 33.1..."
1,28922612,None,None,None,None,12,141921,0.0,0.8,0.2,OH,456,None,0.0,0.8,0.5,Up To 1.00,N,ASH,"POLYGON ((-117.06527 33.13968, -117.06527 33.1..."
2,28922613,None,None,None,None,12,141921,0.0,0.8,0.2,OH,456,None,0.0,0.8,0.5,Up To 1.00,N,ASH,"POLYGON ((-117.06527 33.13934, -117.06527 33.1..."
3,28922614,None,None,None,None,12,139746,0.0,0.0,0.2,OH,456,None,0.7,0.0,0.9,0,N,ASH,"POLYGON ((-117.05465 33.14209, -117.05465 33.1..."
4,28922615,None,None,None,None,12,139746,0.0,0.0,0.2,OH,456,None,0.7,0.0,0.9,0,N,ASH,"POLYGON ((-117.05465 33.14175, -117.05465 33.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477050,29399661,None,None,None,None,12,177379,0.0,0.2,0.1,OH,209,None,0.0,0.2,0.1,Up To 1.00,N,MELROSE,"POLYGON ((-117.2101 33.23231, -117.2101 33.232..."
477051,29399662,None,None,None,None,12,177379,0.0,0.2,0.1,OH,209,None,0.0,0.2,0.1,Up To 1.00,N,MELROSE,"POLYGON ((-117.2101 33.23197, -117.2101 33.232..."
477052,29399663,None,None,None,None,12,179642,0.0,0.2,0.1,OH,209,None,0.0,0.2,0.1,Up To 1.00,N,MELROSE,"POLYGON ((-117.20929 33.23334, -117.20929 33.2..."
477053,29399664,None,None,None,None,12,179642,0.0,0.2,0.1,OH,209,None,0.0,0.2,0.1,Up To 1.00,N,MELROSE,"POLYGON ((-117.20929 33.233, -117.20929 33.233..."


## Electricity Cost

Prices taken from EnergySage. https://www.energysage.com/local-data/electricity-cost/ca/

In [3]:
county_regex = re.compile(rf"The average residential electricity rate in Alameda County, CA is (\d+) ¢/kWh")

matching = county_regex.findall("The average residential electricity rate in Alameda County, CA is 30 ¢/kWh")

for matches in matching:
    print(matches)

30


In [4]:
# get cost for each California county

ca_counties["Electricity Price (cents/kWh)"] = np.nan # initialize the column.

# find EnergySage URL
energy_sage_ca = "https://www.energysage.com/local-data/electricity-cost/ca/"

for idx, county in ca_counties.iterrows():
    print(county["NAME"])

    county_suffix = county["NAME"].replace(" ", "-").lower() + "-county" # convert county name to URL form

    county_energy_sage_url = energy_sage_ca + county_suffix

    county_energy_sage_html = requests.get(county_energy_sage_url) # retrieve website content

    # use regular expressions to extract electricity price
    county_regex = re.compile(rf"The average residential electricity rate in {county['NAME']} County, CA is (\d+) ¢/kWh")

    try:
        county_energy_sage_html = requests.get(county_energy_sage_url)
        matching = county_regex.findall(county_energy_sage_html.text)

        if len(matching) > 1:
            print(f"{county['NAME']} has more than one match.")
        elif len(matching) == 0:
            print(f"{county['NAME']} has no matches.")

        for matches in matching:
            ca_counties.loc[idx, 'Electricity Price (cents/kWh)'] = float(matches)

    except:
        print(county_suffix)

    time.sleep(1)


ca_counties["Electricity Price ($/MWh)"] = ca_counties["Electricity Price (cents/kWh)"]*10 # convert units (cents/kWh) to ($/MWh)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Alameda
Alpine
Alpine has no matches.
Amador
Contra Costa
Fresno
Humboldt
Lake
Los Angeles
Merced
Nevada
San Mateo
Santa Clara
Yuba
Mendocino
Mono
Mono has no matches.
Napa
Orange
San Benito
San Diego
San Luis Obispo
Santa Barbara
Sonoma
Sutter
Tulare
Yolo
Madera
Calaveras
Mariposa
Solano
Plumas
Siskiyou
Inyo
Monterey
Stanislaus
Sierra
Sierra has no matches.
Shasta
Colusa
Colusa has no matches.
Marin
San Joaquin
Kings
Lassen
Lassen has no matches.
San Francisco
Glenn
Glenn has no matches.
Imperial
Placer
Tuolumne
Ventura
Tehama
Modoc
Modoc has no matches.
Santa Cruz
Trinity
Trinity has no matches.
Kern
San Bernardino
El Dorado
Sacramento
Butte
Del Norte
Riverside


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
ca_counties.to_file("Data/Boundaries/ca_county_prices_EnergySage.geojson", driver="GeoJSON")

Try again: Prices Taken from GridStatus.io. https://www.gridstatus.io/datasets/caiso_lmp_day_ahead_hourly

In [21]:
# data source: has data from 2019-03-21 PDT to present.
# let's do 2023 data. (don't do a leap year to avoid confusion about time indexing)

# takes around 1 minute to run

client = GridStatusClient(GRIDSTATUS_API_KEY)

df = client.get_dataset(
    dataset="caiso_lmp_day_ahead_hourly",
    start="2023-01-01 00:00:00",
    end="2023-12-31 01:00:00",
    filter_column="location_type",
    filter_value="AP Node",
    resample="1 year"
)

2025-09-04 23:14:21 - INFO - Fetching Page 1...
2025-09-04 23:14:21 - INFO - GET https://api.gridstatus.io/v1/datasets/caiso_lmp_day_ahead_hourly/query
2025-09-04 23:14:21 - INFO - Params: {'start_time': Timestamp('2023-01-01 00:00:00'), 'end_time': Timestamp('2023-12-31 01:00:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': '1 year', 'resample_by': None, 'resample_function': 'mean', 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'location_type', 'filter_value': 'AP Node', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-09-04 23:15:07 - INFO - Done in 45.64 seconds. 
2025-09-04 23:15:07 - INFO - Total number of rows: 2523


In [23]:
df.to_csv("Data/Prices/gridstatus_caiso_2023.csv")

## Electricity Demand Profile

In [48]:
# CAISO demand profile
client = GridStatusClient(GRIDSTATUS_API_KEY)

df = client.get_dataset(
    dataset="caiso_load_hourly",
    start="2013-01-01",
    end="2014-01-01",
    filter_column='tac_area_name',
    filter_value="CA ISO-TAC"
)

2025-09-05 15:25:14 - INFO - Fetching Page 1...
2025-09-05 15:25:14 - INFO - GET https://api.gridstatus.io/v1/datasets/caiso_load_hourly/query
2025-09-05 15:25:14 - INFO - Params: {'start_time': Timestamp('2013-01-01 00:00:00'), 'end_time': Timestamp('2014-01-01 00:00:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'tac_area_name', 'filter_value': 'CA ISO-TAC', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-09-05 15:25:14 - INFO - Done in 0.3 seconds. 
2025-09-05 15:25:14 - INFO - Total number of rows: 8760


In [49]:
# download data
df.to_csv("Data/Demand Profiles/CAISO_2013_profile.csv")

In [ ]:
# 2011 CAISO demand profile: for validation
client = GridStatusClient(GRIDSTATUS_API_KEY)

df = client.get_dataset(
    dataset="caiso_load_hourly",
    start="2011-01-01",
    end="2012-01-01",
    filter_column='tac_area_name',
    filter_value="CA ISO-TAC"
)

# download data
df.to_csv("Data/Demand Profiles/CAISO_2011_profile.csv")

2025-09-06 21:00:52 - INFO - Fetching Page 1...
2025-09-06 21:00:52 - INFO - GET https://api.gridstatus.io/v1/datasets/caiso_load_hourly/query
2025-09-06 21:00:52 - INFO - Params: {'start_time': Timestamp('2011-01-01 00:00:00'), 'end_time': Timestamp('2012-01-01 00:00:00'), 'publish_time_start': None, 'publish_time_end': None, 'limit': None, 'page': 1, 'page_size': None, 'resample_frequency': None, 'resample_by': None, 'resample_function': None, 'publish_time': None, 'timezone': None, 'cursor': '', 'filter_column': 'tac_area_name', 'filter_value': 'CA ISO-TAC', 'filter_operator': '=', 'return_format': 'json', 'json_schema': 'array-of-arrays'}
2025-09-06 21:00:52 - INFO - Done in 0.42 seconds. 
2025-09-06 21:00:52 - INFO - Total number of rows: 8760


## Solar + Wind Data (Emissions, Water Scarcity, Cost)

#### Capacity Factor

##### NSRDB

Reference: https://github.com/NREL/hsds-examples/tree/master

In [3]:
# takes around a minute to run 

solar = h5pyd.File(f"/nrel/nsrdb/v3/nsrdb_2011.h5")
meta = pd.DataFrame(solar['meta'][...])

In [4]:
# find the nearest timeseries
# code source: https://github.com/NREL/hsds-examples/blob/master/notebooks/03_NSRDB_introduction.ipynb
dset_coords = solar['coordinates'][...]
tree = cKDTree(dset_coords)

def nearest_site_solar(tree, lat_coord, lon_coord):
    """ 
    Find the nearest site in the solar dataset.

    Parameters
    ----------
        tree: scipy.spatial.cKDTree
            Lookup tree for nearest neighbor
        lat_coord: float
            Latitude
        lon_coord: float
            Longitude

    Returns
    -------
        Index of closest point in the solar dataset
    """

    lat_lon = np.array([lat_coord, lon_coord])
    _, pos = tree.query(lat_lon)
    return pos

In [5]:
# collect DNI, DHI, GHI at every point
# takes around 4 min to run

dni_data = solar['dni']
dhi_data = solar['dhi']
ghi_data = solar['ghi']

dni_subbasins = np.zeros((dni_data.shape[0], huc8_ca.shape[0])) # rows: timestamps, # columns: locations
dhi_subbasins = np.zeros((dhi_data.shape[0], huc8_ca.shape[0])) # rows: timestamps, # columns: locations
ghi_subbasins = np.zeros((ghi_data.shape[0], huc8_ca.shape[0])) # rows: timestamps, # columns: locations

time_offset = np.zeros(huc8_ca.shape[0])

# for each California subbasin centroid, get the solar data of the closest point
for idx, (_, row) in enumerate(huc8_ca.iterrows()):
    centroid_idx = nearest_site_solar(tree, row['centroid_lat'], row['centroid_lon'])
    
    time_offset[idx] = meta.loc[centroid_idx, 'timezone']

    dni_tseries = dni_data[:, centroid_idx] / dni_data.attrs['psm_scale_factor']
    dhi_tseries = dhi_data[:, centroid_idx] / dhi_data.attrs['psm_scale_factor']
    ghi_tseries = ghi_data[:, centroid_idx] / ghi_data.attrs['psm_scale_factor'] # units: W/m^2. https://nsrdb.nrel.gov/data-sets/us-data

    dni_subbasins[:, idx] = dni_tseries
    dhi_subbasins[:, idx] = dhi_tseries
    ghi_subbasins[:, idx] = ghi_tseries

In [6]:
# by visual inspection of the data, we can tell that this time index data is UTC timezone. (California is -8 hours, according to the metadata)
# collect time index
time_index = pd.to_datetime(solar['time_index'][...].astype(str))

In [7]:
irradiation_xr = xr.Dataset()
irradiation_xr['DNI'] = xr.DataArray(dni_subbasins, dims=("Time", "HUC8"), coords={"Time": time_index, "HUC8": huc8_ca.index})
irradiation_xr['DHI'] = xr.DataArray(dhi_subbasins, dims=("Time", "HUC8"), coords={"Time": time_index, "HUC8": huc8_ca.index})
irradiation_xr['GHI'] = xr.DataArray(ghi_subbasins, dims=("Time", "HUC8"), coords={"Time": time_index, "HUC8": huc8_ca.index})

irradiation_xr.attrs['Timezone'] = time_offset

In [8]:
# save data
irradiation_xr.to_netcdf("Data/Solar/ca_irradiation_2011.nc4")

# np.savez("Data/Solar/ca_irradiation.npz", dni=dni_subbasins, dhi=dhi_subbasins, ghi=ghi_subbasins, time_index=time_index)

##### WTK

Most common wind turbine models in California: V90-3.0 (489 count), GE1.5-77 (461 count), MM92 (245 count)

Most common wind turbine hub heights in California: 80.0m (1297 count), 65.0m (375 count), 24.0m (281 count)

Let's just fix the wind turbine model at GE1.5-77. Hub height 80.0m

https://en.wind-turbine-models.com/turbines/565-ge-vernova-ge-1.5s

https://nrel.github.io/turbine-models/DOE_GE_1.5MW_77.html

In [2]:
# question: what is typical turbine model, turbine height in California? Let's look at USWTDB: https://energy.usgs.gov/uswtdb/data/

existing_turbines_US = gpd.read_file("Data/Wind/uswtdbGeoJSON/uswtdb_V8_1_20250522.geojson")

# filter to California
existing_turbines_CA = existing_turbines_US[existing_turbines_US['t_state'] == "CA"]

existing_turbines_CA['t_model'].value_counts() # turbine models

existing_turbines_CA['t_hh'].value_counts() # turbine hub heights

t_hh
80.0     1297
65.0      375
24.0      281
60.0      206
78.5      161
24.6      132
55.0       95
67.0       85
50.0       74
91.5       64
84.0       60
85.0       55
94.0       51
64.0       50
69.0       45
68.5       42
40.0       36
92.0       32
82.0       28
81.5       23
90.0       23
89.0       22
105.0      19
63.0       19
45.0       17
87.0       13
39.5       13
30.0        8
56.5        6
Name: count, dtype: int64

Downloading wind speed data

In [9]:
wind = h5pyd.File("/nrel/wtk-us.h5")

wind_speed = wind['windspeed_80m'] # the most common wind speed is 80m among California wind turbines

# note: the wind data is hourly
wind_time = pd.DataFrame({'datetime': wind['datetime']})
wind_time['datetime'] = wind_time['datetime'].apply(dateutil.parser.parse)
wind_time['datetime'] = wind_time['datetime'].map(lambda x: x.replace(tzinfo=timezone.utc))

# get 2011 data when querying for specific locations, right now data is too large
# just select 2011 data
wind_time['year'] = wind_time['datetime'].map(lambda x: x.year)
wind_time = wind_time[wind_time['year'] == 2011]
indices_2011 = wind_time.index

In [10]:
# code source: https://github.com/NREL/hsds-examples/blob/master/notebooks/01_WTK_introduction.ipynb
def nearest_wind_site(data, lat_index, lon_index):
    """ 
    Get nearest site with wind data to input location.

    Parameters
    ----------
        data: h5pyd._hl.dataset.Dataset
            Full dataset, for coordinates
        lat_index: float
            Latitude
        lon_index: float
            Longitude

    Returns
    -------
        Latitude and longitude index of closest point in the wind dataset.
    """

    data_coords = data['coordinates']

    # find nearby point, convert to x/y indices
    projstring = """+proj=lcc +lat_1=30 +lat_2=60 
                    +lat_0=38.47240422490422 +lon_0=-96.0 
                    +x_0=0 +y_0=0 +ellps=sphere 
                    +units=m +no_defs """
    project_lcc = Proj(projstring)
    origin_ll = reversed(data_coords[0][0]) # origin. But flip order to get lon, lat
    origin = project_lcc(*origin_ll)

    coords = (lon_index, lat_index) # note the order of lon, lat
    coords = project_lcc(*coords)
    delta = np.subtract(coords, origin)
    ij = [int(round(x/2000)) for x in delta] # by rounding, we just get the nearest lattice point
    return tuple(reversed(ij))

In [11]:
# get wind speeds at closest location to each of the California HUC8 subbasins

# takes around 30 minutes to run

wind_speed_subbasins = np.zeros((wind_time.shape[0], huc8_ca.shape[0]))

# for each wind subbasin centroid, get wind data of closest point

while np.any(np.sum(wind_speed_subbasins, axis=0) == 0):
    for idx, (_, row) in enumerate(huc8_ca.iterrows()):
        if np.sum(wind_speed_subbasins[:,idx]) == 0:
            try:
                centroid_idx = nearest_wind_site(wind, row['centroid_lat'], row['centroid_lon'])

                print(centroid_idx)

                
                wind_tseries = wind_speed[indices_2011, centroid_idx[0], centroid_idx[1]] # units: m/s.
                
                wind_tseries = wind_speed[indices_2011, centroid_idx[0], centroid_idx[1]] # units: m/s.

                wind_speed_subbasins[:, idx] = wind_tseries
            except:
                pass
    
    time.sleep(10)

(593, 635)
(608, 546)
(613, 565)
(618, 598)
(622, 534)
(632, 565)
(638, 641)
(642, 535)
(655, 538)
(662, 515)
(666, 568)
(669, 542)
(681, 608)
(688, 530)
(685, 658)
(723, 577)
(717, 648)
(745, 557)
(753, 628)
(781, 542)
(674, 506)
(681, 499)
(695, 503)
(705, 476)
(707, 492)
(718, 464)
(733, 451)
(731, 476)
(743, 514)
(767, 481)
(793, 561)
(794, 521)
(788, 587)
(785, 603)
(814, 516)
(829, 537)
(848, 553)
(882, 531)
(884, 550)
(912, 522)
(791, 482)
(803, 482)
(819, 493)
(817, 454)
(836, 491)
(855, 454)
(857, 510)
(883, 465)
(883, 501)
(908, 478)
(923, 477)
(938, 470)
(943, 506)
(958, 499)
(975, 470)
(980, 492)
(990, 472)
(1002, 494)
(1026, 479)
(1043, 483)
(931, 514)
(1065, 518)
(1084, 525)
(1112, 531)
(1132, 539)
(1057, 499)
(1081, 510)
(1109, 502)
(1131, 525)
(1138, 492)
(658, 462)
(718, 417)
(746, 421)
(752, 425)
(767, 423)
(766, 439)
(780, 442)
(807, 427)
(863, 422)
(887, 445)
(766, 409)
(827, 393)
(839, 404)
(883, 405)
(895, 385)
(914, 392)
(921, 373)
(931, 394)
(959, 398)
(971, 367

In [14]:
# save wind speeds to csv

wind_speed_subbasins_df = pd.DataFrame(wind_speed_subbasins, index=wind_time['datetime'], columns=huc8_ca.index)
wind_speed_subbasins_df.to_csv("Data/Wind/ca_wind_speed_2011.csv")